In [61]:
import sys
sys.path.append('..')
import networkx as nx
import numpy as np
import socnet as sn
from scipy.spatial import distance

In [62]:
ug = sn.load_graph('one-mode.gml')
dg = sn.load_graph('two-mode.gml')
sn.node_label_position = 'middle center'
sn.edge_label_distance = 10
sn.graph_width = 1280
sn.graph_height = 720

sn.show_graph(ug, nlab=True)

../socnet.py:59: RuntimeWarning:

divide by zero encountered in double_scalars

../socnet.py:187: RuntimeWarning:

invalid value encountered in double_scalars

../socnet.py:188: RuntimeWarning:

invalid value encountered in double_scalars

../socnet.py:189: RuntimeWarning:

invalid value encountered in double_scalars

../socnet.py:190: RuntimeWarning:

invalid value encountered in double_scalars



In [63]:
sn.show_graph(dg, nlab=True)

In [64]:
lista1 = []


for n in ug.in_degree():
    print(ug.nodes[n[0]]['label'], ': {0} votos'.format(n[1]))
    lista1.append((ug.nodes[n[0]]['label'], n[1]))

votosranking = sorted(lista1, key= lambda x : x[1], reverse = True)

print('\n')
print('\n')

print('Esse são os alunos ordenados por mais votados')
print('\n')
print('\n')

for i in votosranking:
    print("{0} : Votos : {1}".format(i[0],i[1]))




Rachel : 5 votos
Jean : 1 votos
Bruna : 1 votos
Leo : 2 votos
Joao : 0 votos
Rapha : 3 votos
Carlos : 1 votos
Fred : 7 votos
Toyama : 7 votos
Vitoria : 5 votos
Hugo : 12 votos
Gabs : 2 votos
Elisa : 2 votos
Paulo : 0 votos
Sabrina : 2 votos
Forba : 7 votos
Rios : 2 votos
Ale : 0 votos
Graicer : 10 votos
Antonio : 3 votos
Martim : 2 votos
Pedro : 0 votos
Isa : 3 votos
Chen : 6 votos
Tirta : 0 votos




Esse são os alunos ordenados por mais votados




Hugo : Votos : 12
Graicer : Votos : 10
Fred : Votos : 7
Toyama : Votos : 7
Forba : Votos : 7
Chen : Votos : 6
Rachel : Votos : 5
Vitoria : Votos : 5
Rapha : Votos : 3
Antonio : Votos : 3
Isa : Votos : 3
Leo : Votos : 2
Gabs : Votos : 2
Elisa : Votos : 2
Sabrina : Votos : 2
Rios : Votos : 2
Martim : Votos : 2
Jean : Votos : 1
Bruna : Votos : 1
Carlos : Votos : 1
Joao : Votos : 0
Paulo : Votos : 0
Ale : Votos : 0
Pedro : Votos : 0
Tirta : Votos : 0


In [65]:
for n in dg.in_degree():
    print(dg.nodes[n[0]]['label'], ': {0} votos'.format(n[1]))

Samba : 3 votos
Isa : 0 votos
Chen : 0 votos
MPB : 7 votos
Rap Nacional : 10 votos
Pop : 11 votos
Ale : 0 votos
Rapha : 0 votos
Paulo : 0 votos
Elisa : 0 votos
Pedro : 0 votos
Antonio : 0 votos
Forba : 0 votos
Hugo : 0 votos
K-Pop : 2 votos
Metal : 7 votos
Vitoria : 0 votos
Indie Rock : 8 votos
Fred : 0 votos
Funk : 9 votos
Rachel : 0 votos
Toyama : 0 votos
Sertanejo : 8 votos
Sabrina : 0 votos
Eletronica : 19 votos
Pop Rock : 11 votos
Disco : 8 votos
Gabs : 0 votos
Opera : 2 votos
Bruna : 0 votos
Martim : 0 votos
Pagode : 10 votos
Rock classico : 11 votos
Joao : 0 votos
Jean : 0 votos
Carlos : 0 votos
Jazz : 6 votos
J-Pop : 7 votos
Leo : 0 votos
Graicer : 0 votos


In [66]:
e = 0.0001

In [67]:
def get_norm(v):
    norm=np.linalg.norm(v, ord=1)
    if norm==0:
        norm=np.finfo(v.dtype).eps
    return norm


In [68]:
#preparando o terreno
alunos = []
alunos_votes_destination = []
generos = []
index = 0
for n in dg.nodes:
    if(len(list(dg.successors(index))) == 0):
        generos.append(index)
    else:
        alunos.append(index)
        alunos_votes_destination.append(list(dg.successors(index)))
    index+=1

In [69]:
def ping_pong(firstTime, lastNormalizedPoints = [], lastNormalizedImportance =[]):
    #doing stuff
    if(firstTime): #ainda n exsite points nos nós de genero
        for n in generos:
            dg.nodes[n]['points']  = dg.in_degree()[n]
    else:
        for i in generos:
            new_point = 0
            for j in dg.predecessors(i):
                new_point +=  dg.nodes[j]['importance']
            dg.nodes[i]['points']  = new_point
        

    iterator = 0
    for i in alunos:
        importance = 0
        for j in alunos_votes_destination[iterator]:
            importance += dg.nodes[j]['points'] 
        iterator+=1
        dg.nodes[i]['importance'] = importance
        
    # normalizando pontos das cagetorias
    points_to_normalize = []
    for i in generos:
        points_to_normalize.append(dg.nodes[i]['points'])

    norm = get_norm(points_to_normalize)
    normalized_points = []
    for i in generos:
        normalized_points.append( dg.nodes[i]['points']/norm)
        dg.nodes[i]['normalized_points'] = dg.nodes[i]['points']/norm
        
    #normalizando importancia dos alunos
    importance_to_normalize = []
    
    for i in alunos:
        importance_to_normalize.append(dg.nodes[i]['importance'])

    norm = get_norm(importance_to_normalize)
    normalized_importance = []
    for i in alunos:
        normalized_importance.append(dg.nodes[i]['importance']/norm)
        dg.nodes[i]['normalized_importance'] = dg.nodes[i]['importance']/norm
    
    if(firstTime):
        ping_pong(False,normalized_points, normalized_importance)
    else:
        distPoints = distance.euclidean(normalized_points, lastNormalizedPoints)
        distImportance = distance.euclidean(normalized_importance, lastNormalizedImportance)
        if((distPoints <= e) and (distImportance <= e)):
            return
        else:
            ping_pong(False,normalized_points, normalized_importance)
        

In [70]:
ping_pong(True)

Ao final das iterações chegamos ao seguinte grau de importância dos votos por pessoa.

In [71]:
lista = []
for n in alunos:
    print("{0} : Importância : {1}".format(dg.nodes[n]['label'],dg.nodes[n]['importance']))
    lista.append((dg.nodes[n]['label'],dg.nodes[n]['importance'],n))
    
importanciaranking = sorted(lista, key= lambda x: x[1], reverse=True)
print('\n')
print('\n')

print('Esse são os alunos ordenados por odem de importância de voto')
print('\n')
print('\n')

for i in importanciaranking:
    print("{0} : Importância : {1}".format(i[0],i[1]))



Isa : Importância : 137264381467
Chen : Importância : 119528361280
Ale : Importância : 104232116374
Rapha : Importância : 104860209324
Paulo : Importância : 57613782353
Elisa : Importância : 41956113296
Pedro : Importância : 114079444063
Antonio : Importância : 65957077177
Forba : Importância : 256946041422
Hugo : Importância : 139363885359
Vitoria : Importância : 146013741183
Fred : Importância : 120677786445
Rachel : Importância : 146816322620
Toyama : Importância : 162973181670
Sabrina : Importância : 185966229380
Gabs : Importância : 104860209324
Bruna : Importância : 81883621703
Martim : Importância : 111324624983
Joao : Importância : 126925779032
Jean : Importância : 189546070461
Carlos : Importância : 55632171362
Leo : Importância : 14025444748
Graicer : Importância : 128340396868




Esse são os alunos ordenados por odem de importância de voto




Forba : Importância : 256946041422
Jean : Importância : 189546070461
Sabrina : Importância : 185966229380
Toyama : Importância : 162

In [81]:
lista = []
for n in generos:
    print("{0} : Pontos : {1}".format(dg.nodes[n]['label'],dg.nodes[n]['points']))
    lista.append((dg.nodes[n]['label'],dg.nodes[n]['points'],n))
    
generosranking = sorted(lista, key= lambda x: x[1], reverse=True)
print('\n')
print('\n')

print('Esse são os generos ordenados por pontuação')
print('\n')
print('\n')

for i in generosranking:
    print("{0} : Pontos : {1}".format(i[0],i[1]))


Samba : Pontos : 7135990954
MPB : Pontos : 15038602678
Rap Nacional : Pontos : 18633906244
Pop : Pontos : 20467524898
K-Pop : Pontos : 6138376126
Metal : Pontos : 14059624247
Indie Rock : Pontos : 17729184790
Funk : Pontos : 15569958979
Sertanejo : Pontos : 15685480566
Eletronica : Pontos : 35164646464
Pop Rock : Pontos : 21488588398
Disco : Pontos : 17631454703
Opera : Pontos : 2358682907
Pagode : Pontos : 19806217071
Rock classico : Pontos : 22065569708
Jazz : Pontos : 11875429827
J-Pop : Pontos : 14025444748




Esse são os generos ordenados por pontuação




Eletronica : Pontos : 35164646464
Rock classico : Pontos : 22065569708
Pop Rock : Pontos : 21488588398
Pop : Pontos : 20467524898
Pagode : Pontos : 19806217071
Rap Nacional : Pontos : 18633906244
Indie Rock : Pontos : 17729184790
Disco : Pontos : 17631454703
Sertanejo : Pontos : 15685480566
Funk : Pontos : 15569958979
MPB : Pontos : 15038602678
Metal : Pontos : 14059624247
J-Pop : Pontos : 14025444748
Jazz : Pontos : 1187542982

Com base no que temos até agora, vamos descobrir quem são nossos principais candidatos até então.

In [77]:
print('Top 5 DJs mais votados')
print('\n')
for i in votosranking[:5]:
    print(i[0] + ' : ' +  str(i[1]) + ' votos')

Top 5 DJs mais votados


Hugo : 12 votos
Graicer : 10 votos
Fred : 7 votos
Toyama : 7 votos
Forba : 7 votos


In [79]:
print('Top 5 authorities')
print('\n')
for i in importanciaranking[:5]:
    print(i[0] + ' : ' +  str(i[1]) + ' importancia')

Top 5 authorities


Forba : 256946041422 importancia
Jean : 189546070461 importancia
Sabrina : 185966229380 importancia
Toyama : 162973181670 importancia
Rachel : 146816322620 importancia


In [82]:
print('Top 5 hubs')
print('\n')
for i in generosranking[:5]:
    print(i[0] + ' : ' +  str(i[1]) + ' pontos')

Top 5 hubs


Eletronica : 35164646464 pontos
Rock classico : 22065569708 pontos
Pop Rock : 21488588398 pontos
Pop : 20467524898 pontos
Pagode : 19806217071 pontos


# Minha terceira métrica (Betwenness do One-Mode)

A escolha dessa métrica se deve pelo fato de que as maiores pontuações de betweenness são os caras que provavelmente tem mais chancede agradar o todo, uma vez que varios caminhos passam por ele. O uso só do betweenness pode ser um resultado muito "em cima do muro" pois estaríamos elegendo alguem que em média não vai desapontar muita gente. Entretanto, usando o betweenness como UMA das métricas de escolha temos uma decisão mais embasada.

In [84]:
bc = nx.betweenness_centrality(ug)

lista = []
for i in ug.nodes:
    print(ug.nodes[i]['label'] + ' :{0} betweenness'.format(bc[i]))
    lista.append((ug.nodes[i]['label'] ,bc[i],n))
    
betranking = sorted(lista, key= lambda x: x[1], reverse=True)
print('\n')
print('\n')

print('Esse são os djs ordenados por betweenness')
print('\n')
print('\n')

for i in betranking:
    print("{0} : Betweenness : {1}".format(i[0],i[1]))

Rachel :0.059541062801932366 betweenness
Jean :0.002536231884057971 betweenness
Bruna :0.1570350241545894 betweenness
Leo :0.005434782608695652 betweenness
Joao :0.0 betweenness
Rapha :0.021739130434782608 betweenness
Carlos :0.0 betweenness
Fred :0.19716183574879226 betweenness
Toyama :0.05229468599033816 betweenness
Vitoria :0.20410628019323673 betweenness
Hugo :0.08725845410628019 betweenness
Gabs :0.020833333333333332 betweenness
Elisa :0.0 betweenness
Paulo :0.0 betweenness
Sabrina :0.18985507246376815 betweenness
Forba :0.19396135265700482 betweenness
Rios :0.0 betweenness
Ale :0.0 betweenness
Graicer :0.06648550724637682 betweenness
Antonio :0.03804347826086957 betweenness
Martim :0.04528985507246377 betweenness
Pedro :0.0 betweenness
Isa :0.05235507246376812 betweenness
Chen :0.2002717391304348 betweenness
Tirta :0.0 betweenness




Esse são os djs ordenados por betweenness




Vitoria : Betweenness : 0.20410628019323673
Chen : Betweenness : 0.2002717391304348
Fred : Betweennes

Top 5 betweenness

In [85]:
print('Top 5 "intermediários"')
print('\n')
for i in betranking[:5]:
    print("{0} : Betweenness : {1}".format(i[0],i[1]))

Top 5 "intermediários"


Vitoria : Betweenness : 0.20410628019323673
Chen : Betweenness : 0.2002717391304348
Fred : Betweenness : 0.19716183574879226
Forba : Betweenness : 0.19396135265700482
Sabrina : Betweenness : 0.18985507246376815


## Conclusão Parcial


### Pelo fato do Forba ser o único aluno à aparecer bem ranqueado em TODAS as métricas, a conclusão é de que ele é a pessoa indicada para tocar na festa da sala

# Entretanto

Refletindo no ônibus percebi que fazia sentido considerar a pontuação relativa de todo mundo nos 3 critérios, e dar pesos para esses critérios. Por que ? Porque a informação intríseca nos votos é muito mais ricas do que os parâmetros obtidos com os outros 2 critérios, por tanto esse critério terá peso 2. Sendo assim segue a nova pontuação:

In [92]:
def get_pesos(tupleItens):
    total = 0
    for i in tupleItens:
        total += i[1]
    new_tuple_itens = []
    for i in tupleItens:
        new_tuple_itens.append((i[0], i[1]/total))
    return new_tuple_itens

importanciarankingpeso = get_pesos(importanciaranking)
betrankingpeso = get_pesos(betranking)
votosrankingpeso = get_pesos(votosranking)

final_list = {}

for i in importanciarankingpeso:
    if(i[0] in final_list):
        final_list[i[0]] += i[1]
    else:
        final_list[i[0]] = i[1]
        
for i in betrankingpeso:
    if(i[0] in final_list):
        final_list[i[0]] += i[1]
    else:
        final_list[i[0]] = i[1]
        
for i in votosrankingpeso:
    if(i[0] in final_list):
        final_list[i[0]] += i[1]
    else:
        final_list[i[0]] = i[1]*2
        
print(final_list)
        

        


{'Vitoria': 0.24201627583145988, 'Paulo': 0.021206587975023658, 'Rapha': 0.0883780824824845, 'Toyama': 0.1771278509879385, 'Pedro': 0.041990573572155486, 'Carlos': 0.03252538569345482, 'Isa': 0.11951001578145182, 'Joao': 0.046719076398225125, 'Leo': 0.03266798819393478, 'Hugo': 0.25061047940568504, 'Jean': 0.08340757139108908, 'Chen': 0.24191037776442864, 'Tirta': 0.0, 'Antonio': 0.08428581810100552, 'Rios': 0.024096385542168676, 'Sabrina': 0.21163808961069733, 'Fred': 0.25243089550852693, 'Forba': 0.30058119204037975, 'Elisa': 0.03953966962045058, 'Ale': 0.03836595091370593, 'Gabs': 0.07576170789321834, 'Martim': 0.09348205110181412, 'Bruna': 0.1406918561525575, 'Rachel': 0.15162987331505628, 'Graicer': 0.20942624472308752}


In [93]:
import operator
sorted_x = sorted(final_list.items(), key=operator.itemgetter(1))
print(sorted_x)

[('Tirta', 0.0), ('Paulo', 0.021206587975023658), ('Rios', 0.024096385542168676), ('Carlos', 0.03252538569345482), ('Leo', 0.03266798819393478), ('Ale', 0.03836595091370593), ('Elisa', 0.03953966962045058), ('Pedro', 0.041990573572155486), ('Joao', 0.046719076398225125), ('Gabs', 0.07576170789321834), ('Jean', 0.08340757139108908), ('Antonio', 0.08428581810100552), ('Rapha', 0.0883780824824845), ('Martim', 0.09348205110181412), ('Isa', 0.11951001578145182), ('Bruna', 0.1406918561525575), ('Rachel', 0.15162987331505628), ('Toyama', 0.1771278509879385), ('Graicer', 0.20942624472308752), ('Sabrina', 0.21163808961069733), ('Chen', 0.24191037776442864), ('Vitoria', 0.24201627583145988), ('Hugo', 0.25061047940568504), ('Fred', 0.25243089550852693), ('Forba', 0.30058119204037975)]


In [94]:
reversedList = sorted(sorted_x, key= lambda x: x[1], reverse=True)
for i in reversedList:
    print("{0} : Pontuação final : {1}".format(i[0],i[1]))

Forba : Pontuação final : 0.30058119204037975
Fred : Pontuação final : 0.25243089550852693
Hugo : Pontuação final : 0.25061047940568504
Vitoria : Pontuação final : 0.24201627583145988
Chen : Pontuação final : 0.24191037776442864
Sabrina : Pontuação final : 0.21163808961069733
Graicer : Pontuação final : 0.20942624472308752
Toyama : Pontuação final : 0.1771278509879385
Rachel : Pontuação final : 0.15162987331505628
Bruna : Pontuação final : 0.1406918561525575
Isa : Pontuação final : 0.11951001578145182
Martim : Pontuação final : 0.09348205110181412
Rapha : Pontuação final : 0.0883780824824845
Antonio : Pontuação final : 0.08428581810100552
Jean : Pontuação final : 0.08340757139108908
Gabs : Pontuação final : 0.07576170789321834
Joao : Pontuação final : 0.046719076398225125
Pedro : Pontuação final : 0.041990573572155486
Elisa : Pontuação final : 0.03953966962045058
Ale : Pontuação final : 0.03836595091370593
Leo : Pontuação final : 0.03266798819393478
Carlos : Pontuação final : 0.0325253

## Conclusão final:

## Os 3 melhores DJ para a festa da sala são respectivamente: O Forba, o Fred e O Hugo (eu).